In [1]:
import json
from enum import Enum
from collections import namedtuple
from ipynb.fs.full.Critical_Section import *
from ipynb.fs.full.CPv1 import *
import numpy as np
from copy import copy 

In [2]:
class lock_event:
    def __init__(self,n_thr):
        self.lock_clock = np.zeros(n_thr)
        self.objects = {}

class lock_map:
    def __init__(self,init_vc):
        self.lock_vc = init_vc
        self.read_objects = {}
        self.write_objects = {}
    
    def __str__(self):
        return "Main Clock : %s\nObject Read Clocks : %s\nWrite Clocks : %s\n" % (self.lock_vc,self.read_objects,self.write_objects)
    
class object_vc:
    def __init__(self,l_vc,tid):
        self.vc = l_vc
        self.tid = tid

In [86]:
def vc_cp_ordered(vc1,vc2): # Checks if vc1 CPs vc2. Returns true if vc2 does not CP vc1
    r = False
    ct = 0
    for i in range(len(vc1)):
        if vc2[i] > vc1[i]:
            return True
        elif vc2[i] == vc1[i]:
            ct+=1
    return ct==len(vc1)

def compare_vc_to_other_locks(evt,compare_map,v_vc,t_stk):
    ret = False
    for k in compare_map.keys():
        if k == evt.var_name: # Check if currently accessed under any lock and earlier accessed whithout any lock
#             print("Empty",v_vc,compare_map[evt.var_name])
            if evt.tid != compare_map[evt.var_name].tid:
                ret = ret or vc_cp_ordered(v_vc,compare_map[evt.var_name].vc)
        elif isinstance(compare_map[k],lock_map) and k not in t_stk:
            if (evt.e_type == events_list.Read.value and evt.var_name in compare_map[k].write_objects.keys()) or evt.var_name in compare_map[k].write_objects.keys():
#                 print("Lock",v_vc,compare_map[k].write_objects[evt.var_name].vc,t_stk,k)
                if evt.tid != compare_map[k].write_objects[evt.var_name].tid:
                    ret = ret or vc_cp_ordered(v_vc,compare_map[k].write_objects[evt.var_name].vc)
            elif evt.e_type == events_list.Write.value and evt.var_name in compare_map[k].read_objects.keys():
                if evt.tid != compare_map[k].read_objects[evt.var_name].tid:
                    ret = ret or vc_cp_ordered(v_vc,compare_map[k].read_objects[evt.var_name].vc)
    return ret

def add_current_vc(evt, t_map, t_vc, l_stk,s_l_map):
#     if evt.var_name in l_map.keys():
#         l_map[evt.var_name].append(t_vc)
#     else:
    if len(l_stk) == 0:
        s_l_map[evt.var_name] = object_vc(t_vc,evt.tid) #t_vc
#         print(evt.var_name," : ",l_map[evt.var_name])
    else:
        if evt.e_type == events_list.Read.value:
            s_l_map[l_stk[-1]].read_objects[evt.var_name] = object_vc(t_vc,evt.tid)
            t_map.read_objects[evt.var_name] = object_vc(t_vc,evt.tid)
        elif evt.e_type == events_list.Write.value:
            s_l_map[l_stk[-1]].write_objects[evt.var_name] = object_vc(t_vc,evt.tid)
            t_map.write_objects[evt.var_name] = object_vc(t_vc,evt.tid)
#         print(l_stk[-1]+" : ")
#         print(l_map)
        
def get_vc_var(v_name,t_vc,l_map,t_stk):
    if len(t_stk) == 0:
        return t_vc
    else:
        return l_map.lock_vc

### Read the events from log

In [87]:
# events, critical_section_list,num_threads,lock_access_list = gen_event_cs2("temp.log")
# cp_matrix_v2 = pass2(events,critical_section_list,lock_access_list)

# for x in critical_section_list:
#     print(x)

# print(num_threads,lock_access_list)

In [88]:
def update_clocks_temp(t_map,main_map,e_idx,cs_idx_list):
    for cs_idx in cs_idx_list:
        if e_idx in critical_section_list[cs_idx].event_idx:
            for x in critical_section_list[cs_idx].read_objects:
                if x in main_map.read_objects.keys():
                    t_map.read_objects[x] = main_map.read_objects[x]
                else:
                    t_map.read_objects[x] = object_vc(t_map.lock_vc,events[e_idx].tid)
            for x in critical_section_list[cs_idx].write_objects:
                if x in main_map.write_objects.keys():
                    t_map.write_objects[x] = main_map.write_objects[x]
                else:
                    t_map.write_objects[x] = object_vc(t_map.lock_vc,events[e_idx].tid)
                    
def check_rule1(evt,evt_idx):
    cs_idx_list = 0
    for cs_idx in lock_access_list[evt.var_name]:
        if evt_idx in critical_section_list[cs_idx].event_idx:
            current_cs_idx = cs_idx
            break
        cs_idx_list+=1
    
    if cs_idx_list>0:
        if is_cp(cs_idx,lock_access_list[evt.var_name][cs_idx_list-1],critical_section_list) or critical_section_list[cs_idx].tid == critical_section_list[lock_access_list[evt.var_name][cs_idx_list-1]].tid:
            return True

def merge_maps(map_to,map_from):
    for k in map_from.read_objects.keys():
        map_to.read_objects[k] = map_from.read_objects[k]
    for k in map_from.write_objects.keys():
        map_to.write_objects[k] = map_from.write_objects[k]
    map_to.lock_vc = map_from.lock_vc

In [113]:
def CP_V2(num_threads,lock_access_list):
    thread_lock_stack = []
    thread_map_stack = []
    thread_vc = np.zeros((num_threads,num_threads))
    for i in range(num_threads):
        thread_lock_stack.append([])
        thread_map_stack.append([])
    evt_idx = 0
    super_locks_map = {}

    for e in events:
        temp_map = None
#         print("Index : ",evt_idx)
        if e.e_type == events_list.Read.value or e.e_type == events_list.Write.value:
            if len(thread_lock_stack[e.tid]) == 0:
                var_vc = get_vc_var(e.var_name,thread_vc[e.tid],super_locks_map,thread_lock_stack[e.tid])
                if compare_vc_to_other_locks(e,super_locks_map,var_vc,thread_lock_stack[e.tid]):
                    print("DataRace at: \nEvent Index :",evt_idx,"\tThread :",e.tid,"\tEvent Type :",e.e_type," \tVariable : ",e.var_name)
                add_current_vc(e,None,var_vc,thread_lock_stack[e.tid],super_locks_map)
            else:
                temp_map = thread_map_stack[e.tid][-1]
                var_vc = get_vc_var(e.var_name,thread_vc[e.tid],temp_map,thread_lock_stack[e.tid])
#                 print(temp_map,var_vc)
                if compare_vc_to_other_locks(e,super_locks_map,var_vc,thread_lock_stack[e.tid]):
                    print("DataRace at: \nEvent Index :",evt_idx,"\tThread :",e.tid,"\tEvent Type :",e.e_type," \tVariable : ",e.var_name)
                add_current_vc(e,temp_map,var_vc,thread_lock_stack[e.tid],super_locks_map)
                thread_map_stack[e.tid][-1] = temp_map
        elif e.e_type == events_list.Acquire.value:
            thread_lock_stack[e.tid].append(e.var_name)
            if e.var_name not in super_locks_map.keys():
                temp_map = lock_map(thread_vc[e.tid])#super_locks_map[e.var_name]
                super_locks_map[e.var_name] = temp_map
                update_clocks_temp(temp_map,super_locks_map[e.var_name],evt_idx,lock_access_list[e.var_name])
            else:
                temp_map = lock_map(super_locks_map[e.var_name].lock_vc.copy())
                update_clocks_temp(temp_map,super_locks_map[e.var_name],evt_idx,lock_access_list[e.var_name])
            if len(thread_lock_stack[e.tid]) > 1: # Update the lock VC of the child clock to parents clock
                if vc_cp_ordered(temp_map.lock_vc,super_locks_map[thread_lock_stack[e.tid][-2]].lock_vc):
                    temp_map.lock_vc = super_locks_map[thread_lock_stack[e.tid][-2]].lock_vc.copy()
            
            if check_rule1(e,evt_idx):
                temp_map.lock_vc[e.tid]=temp_map.lock_vc[e.tid]+1 # cp_clock_increment
            thread_map_stack[e.tid].append(temp_map)
#             print("Temp Map : ",e.var_name)
#             print(temp_map)
        elif e.e_type == events_list.Release.value:
            popped_lock = thread_lock_stack[e.tid].pop()
            if popped_lock in super_locks_map:
                merge_maps(super_locks_map[popped_lock],thread_map_stack[e.tid].pop())
            else:
                super_locks_map[popped_lock] = thread_map_stack[e.tid].pop()

            if len(thread_lock_stack[e.tid]) > 0:
                merge_maps(thread_map_stack[e.tid][-1],super_locks_map[popped_lock])
            else:
                if not vc_cp_ordered(super_locks_map[popped_lock].lock_vc,thread_vc[e.tid]):
                    thread_vc[e.tid] = super_locks_map[popped_lock].lock_vc
        evt_idx+=1
#         print("Locks Map : ")
#         for k in super_locks_map.keys():
#             print(k," : ",super_locks_map[k])
#         print("\n")

In [114]:
# events, critical_section_list,num_threads,lock_access_list = gen_event_cs2("TestCases/Figure9a.log")
# events, critical_section_list,num_threads,lock_access_list = gen_event_cs2("temp.log")
events, critical_section_list,num_threads,lock_access_list = gen_event_cs2("TestCases/CP_Paper_TC/Figure5.log")
# for x in critical_section_list:
#     print(x)
# # print(num_threads,lock_access_list)
CP_V2(num_threads,lock_access_list)

In [117]:
import glob
for filename in glob.glob("TestCases/*.log"):
    print(filename,)
    events, critical_section_list,num_threads,lock_access_list = gen_event_cs2(filename)
    CP_V2(num_threads,lock_access_list)

TestCases/Figure1.log
DataRace at: 
Event Index : 7 	Thread : 2 	Event Type : 0  	Variable :  test/Figure1_x
TestCases/TestCase2.log
DataRace at: 
Event Index : 1 	Thread : 2 	Event Type : 1  	Variable :  phi_x
TestCases/TestCase6.log
DataRace at: 
Event Index : 4 	Thread : 2 	Event Type : 1  	Variable :  phi_x
TestCases/TestCase4.log
DataRace at: 
Event Index : 2 	Thread : 2 	Event Type : 1  	Variable :  phi_x
TestCases/Figure3.log
TestCases/Figure2_2.log
TestCases/TestCase5.log
TestCases/Figure4b.log
DataRace at: 
Event Index : 33 	Thread : 3 	Event Type : 0  	Variable :  test/Figure4b_x
TestCases/Test.log
DataRace at: 
Event Index : 200 	Thread : 2 	Event Type : 0  	Variable :  test/Test_y
DataRace at: 
Event Index : 400 	Thread : 0 	Event Type : 0  	Variable :  test/Test_y
TestCases/TestCase7.log
DataRace at: 
Event Index : 5 	Thread : 2 	Event Type : 1  	Variable :  phi_x
TestCases/TestCase3.log
TestCases/Figure9a.log
DataRace at: 
Event Index : 16 	Thread : 3 	Event Type : 1  	Va

In [116]:
for filename in glob.glob("TestCases/CP_Paper_TC/*.log"):
    print(filename,"--------------------------------------------------------------------------")
    events, critical_section_list,num_threads,lock_access_list = gen_event_cs2(filename)
    CP_V2(num_threads,lock_access_list)

TestCases/CP_Paper_TC/Figure3a.log --------------------------------------------------------------------------
DataRace at: 
Event Index : 8 	Thread : 2 	Event Type : 0  	Variable :  count
TestCases/CP_Paper_TC/Figure1.log --------------------------------------------------------------------------
TestCases/CP_Paper_TC/Figure9.log --------------------------------------------------------------------------
DataRace at: 
Event Index : 8 	Thread : 2 	Event Type : 1  	Variable :  x
TestCases/CP_Paper_TC/Figure3b.log --------------------------------------------------------------------------
DataRace at: 
Event Index : 5 	Thread : 2 	Event Type : 0  	Variable :  count
TestCases/CP_Paper_TC/Figure6.log --------------------------------------------------------------------------
TestCases/CP_Paper_TC/Figure5.log --------------------------------------------------------------------------
TestCases/CP_Paper_TC/Figure7.log --------------------------------------------------------------------------
TestC